In [1]:
import pandas as pd
import numpy as np
import re

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.metrics import classification_report

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

C:\Users\esteb\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
def add_shifts(df_, col_id, col_calc_pl, new_col, shift):
    for id_ in df_[col_id].unique():
        df_by_id = df_[df_[col_id] == id_]
        df_.loc[df_[col_id] == id_, new_col] = df_by_id[col_calc_pl] - df_by_id[col_calc_pl].shift(shift)
        
    return df_

In [3]:
def get_non_42_cols(df_):
    return [elem for elem in df_.columns if (re.search(r'\d+$', elem) is not None) and (int(elem[-2:].strip()) != 42)]

In [4]:
def interpolate_nan_values(df_):
    list_df = []
    for tick in df_['ticker'].unique():
        df_by_ticker = df_[df_['ticker'] == tick]
        float_columns = df_by_ticker.select_dtypes(float).columns
        df_by_ticker[float_columns] = df_by_ticker[float_columns].interpolate(method='pad', axis=1)
        list_df.append(df_by_ticker)
    return pd.concat(list_df)

In [5]:
df_categorical = pd.read_csv('../data/db_bsm_categorical.csv')

In [6]:
df_financial = pd.read_csv('../data/db_bsm_financial.csv')

In [7]:
list_cols_to_drop = get_non_42_cols(df_financial)

In [8]:
df_financial = df_financial.drop(list_cols_to_drop, 1)

In [9]:
df_financial = df_financial.replace(0, np.nan)

In [10]:
df_financial_no_empty = interpolate_nan_values(df_financial)

C:\Users\esteb\Anaconda3\lib\site-packages\pandas\core\frame.py:3140: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [11]:
df_financial_no_empty = add_shifts(df_financial_no_empty, 'ticker', 'close', 'daily_pl', 1)

In [12]:
df_categorical = df_categorical.dropna()

In [13]:
df_categorical = df_categorical.drop_duplicates(subset=['ticker'], keep='first')

In [14]:
df_fin_cat = pd.merge(left=df_financial_no_empty, right=df_categorical, how='inner', on='ticker')

In [17]:
df_fin_cat = df_fin_cat.dropna()

In [18]:
target = df_fin_cat['sector_gics'].values
features = df_fin_cat.select_dtypes(float).values

In [17]:
# features_rs = RobustScaler().fit_transform(features)
# features_ss = StandardScaler().fit_transform(features)

In [19]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.1, random_state=32)

In [18]:
# X_train_rs, X_test_rs, y_train_rs, y_test_rs = train_test_split(features_rs, target, test_size=0.1, random_state=32)
# X_train_ss, X_test_ss, y_train_ss, y_test_ss = train_test_split(features_ss, target, test_size=0.1, random_state=32)

In [21]:
knn = GridSearchCV(KNeighborsClassifier(),
                   param_grid={'n_neighbors': range(1, 11)}, 
                   scoring='recall_macro', 
                   cv=5)
random_forest = GridSearchCV(RandomForestClassifier(criterion='entropy'),
                   param_grid={'max_depth': range(1, 11)}, 
                   scoring='recall_macro', 
                   cv=5)
dec_tree = GridSearchCV(DecisionTreeClassifier(criterion='entropy'),
                   param_grid={'max_depth': range(1, 11)}, 
                   scoring='recall_macro', 
                   cv=5)
xgb = GridSearchCV(XGBClassifier(n_estimators=100),
                   param_grid={'max_depth': range(1, 11),
                               'learning_rate': [0.001, 0.01, 0.1]}, 
                   scoring='recall_macro', 
                   cv=5)

In [ ]:
%time knn.fit(X_train, y_train)
%time random_forest.fit(X_train, y_train)
%time dec_tree.fit(X_train, y_train)
%time xgb.fit(X_train, y_train)

Wall time: 12min
Wall time: 18min 33s
Wall time: 15min 56s


C:\Users\esteb\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\esteb\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\esteb\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\esteb\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but i

In [ ]:
knn_best = KNeighborsClassifier(**knn.best_params_)
knn_best.fit(X_train, y_train)
print(classification_report(y_test, knn_best.predict(X_test)))

In [ ]:
random_forest_best = RandomForestClassifier(**random_forest.best_params_)
random_forest_best.fit(X_train, y_train)
print(classification_report(y_test, random_forest_best.predict(X_test)))

In [ ]:
dec_tree_best = DecisionTreeClassifier(**dec_tree.best_params_)
dec_tree_best.fit(X_train, y_train)
print(classification_report(y_test, dec_tree_best.predict(X_test)))

In [ ]:
xgb_best = XGBClassifier(**xgb.best_params_)
xgb_best.fit(X_train, y_train)
print(classification_report(y_test, xgb_best.predict(X_test)))

In [20]:
knn = KNeighborsClassifier(**knn.best_params_)
knn.fit(X_train, y_train)

# knn_rs = KNeighborsClassifier(**knn_rs.best_params_)
# knn_rs.fit(X_train_rs, y_train_rs)

# knn_ss = KNeighborsClassifier(**knn_ss.best_params_)
# knn_ss.fit(X_train_ss, y_train_ss)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')

In [21]:
print(classification_report(y_test, knn.predict(X_test)))

NameError: name 'x_test' is not defined

In [24]:
random_forest = GridSearchCV(RandomForestClassifier(criterion='entropy'),
                   param_grid={'max_depth': range(1, 11)}, 
                   scoring='recall_macro', 
                   cv=3)
# random_forest_rs = GridSearchCV(RandomForestClassifier(criterion='entropy'),
#                    param_grid={'max_depth': range(1, 21)}, 
#                    scoring='recall_macro', 
#                    cv=5)
# random_forest_ss = GridSearchCV(RandomForestClassifier(criterion='entropy'),
#                    param_grid={'max_depth': range(1, 21)}, 
#                    scoring='recall_macro', 
#                    cv=5)

In [25]:
%time random_forest.fit(X_train, y_train)
random_forest = RandomForestClassifier(**random_forest.best_params_)
random_forest.fit(X_train, y_train)
print(classification_report(y_test, random_forest.predict(X_test)))
# random_forest_rs.fit(X_train_rs, y_train_rs)
# print('random_forest_rs done')
# random_forest_ss.fit(X_train_ss, y_train_ss)
# print('random_forest_ss done')

Wall time: 9min 52s
                        precision    recall  f1-score   support

Communication services       0.99      0.96      0.97      1886
Consumer discretionary       0.79      0.81      0.80      6469
      Consumer staples       0.88      0.80      0.84      3755
                Energy       0.87      0.85      0.86      3526
            Financials       0.81      0.82      0.81      5026
            Healthcare       0.84      0.50      0.62      1731
           Industrials       0.57      0.79      0.66      2730
Information technology       0.79      0.76      0.77      3292
             Utilities       0.99      0.95      0.97      1201

           avg / total       0.82      0.81      0.81     29616



In [ ]:
random_forest = RandomForestClassifier(**random_forest.best_params_)
random_forest.fit(X_train, y_train)

# random_forest_rs = RandomForestClassifier(**random_forest_rs.best_params_)
# random_forest_rs.fit(X_train_rs, y_train_rs)

# random_forest_ss = RandomForestClassifier(**random_forest_ss.best_params_)
# random_forest_ss.fit(X_train_ss, y_train_ss)

In [ ]:
print(classification_report(y_test, random_forest.predict(X_test)))

In [ ]:
dec_tree = GridSearchCV(DecisionTreeClassifier(criterion='entropy'),
                   param_grid={'max_depth': range(1, 11)}, 
                   scoring='recall_macro', 
                   cv=3)
# dec_tree_rs = GridSearchCV(DecisionTreeClassifier(criterion='entropy'),
#                    param_grid={'max_depth': range(1, 21)}, 
#                    scoring='recall_macro', 
#                    cv=5)
# dec_tree_ss = GridSearchCV(DecisionTreeClassifier(criterion='entropy'),
#                    param_grid={'max_depth': range(1, 21)}, 
#                    scoring='recall_macro', 
#                    cv=5)

In [ ]:
dec_tree.fit(X_train, y_train)
print('dec_tree done')
# dec_tree_rs.fit(X_train_rs, y_train_rs)
# print('dec_tree_rs done')
# dec_tree_ss.fit(X_train_ss, y_train_ss)
# print('dec_tree_ss done')

In [ ]:
dec_tree = DecisionTreeClassifier(**dec_tree.best_params_)
dec_tree.fit(X_train, y_train)

# dec_tree_rs = DecisionTreeClassifier(**dec_tree_rs.best_params_)
# dec_tree_rs.fit(X_train_rs, y_train_rs)

# dec_tree_ss = DecisionTreeClassifier(**dec_tree_ss.best_params_)
# dec_tree_ss.fit(X_train_ss, y_train_ss)

In [ ]:
print(classification_report(y_test, dec_tree.predict(X_test)))

In [ ]:
print(classification_report)